In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.utils import fetch_spot_history, historical_volatility
from src.mc_pricer import bs_price, mc_price_bs, mc_greeks_finite_differences, mc_delta_pathwise, mc_delta_lr
plt.style.use('seaborn')

In [ ]:
series = fetch_spot_history('SPY', period='2y', verbose=True)
S0 = float(series.iloc[-1])
sigma = historical_volatility(series)
r = 0.01
K = S0
T = 30/252  # ~30 calendar days
S0, sigma, r, K, T

In [ ]:
ns = [2000, 10000, 50000, 200000]
mc_prices = []
mc_ses = []
for n in ns:
    p,se,_ = mc_price_bs(S0, K, r, sigma, T, n_sims=n, option='call', seed=123, antithetic=True)
    mc_prices.append(p); mc_ses.append(se)
bs = bs_price(S0,K,r,sigma,T)
plt.errorbar(ns, mc_prices, yerr=[2*s for s in mc_ses], fmt='o-', label='MC +/- 2SE')
plt.axhline(bs, color='r', linestyle='--', label='BS price')
plt.xscale('log'); plt.xlabel('n_sims'); plt.ylabel('price'); plt.legend(); plt.title('MC convergence')

In [ ]:
n_sims = 100000
fd = mc_greeks_finite_differences(S0,K,r,sigma,T,n_sims=n_sims,option='call',bump_S=0.5,bump_sigma=1e-3,seed=123)
pw = mc_delta_pathwise(S0,K,r,sigma,T,n_sims=n_sims,option='call',seed=123)
lr = mc_delta_lr(S0,K,r,sigma,T,n_sims=n_sims,option='call',seed=123)
df = pd.DataFrame({
    'estimator': ['finite_diff','pathwise','likelihood_ratio'],
    'delta': [fd['delta'], pw['delta_pw'], lr['delta_lr']],
    'uncertainty_info': [fd['price_se'], pw['delta_pw_se'], lr['delta_lr_se']]
})
df

In [ ]:
p1,se1,_ = mc_price_bs(S0,K,r,sigma,T,n_sims=50000,seed=123,antithetic=False)
p2,se2,_ = mc_price_bs(S0,K,r,sigma,T,n_sims=50000,seed=123,antithetic=True)
print("No antithetic:", p1, "SE", se1)
print("With antithetic:", p2, "SE", se2)